In [1]:
import numpy as np
import sys
import scipy.stats
import re

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
sys.path.append('../codes')

In [4]:
%pprint

Pretty printing has been turned OFF


In [5]:
from VMSfunctions.Common import *
from VMSfunctions.Chemicals import *
from VMSfunctions.Chromatograms import *
from VMSfunctions.MassSpec import *
from VMSfunctions.Controller import *

# Mass Spec test

In [6]:
import pickle
hmdb = pickle.load(open('../../../Compounds/hmdb_compounds.p','rb'))

In [7]:
ps = load_obj('../models/peak_sampler_4_beers.p')
xcms_output = '../models/beer_ms1_peaks.csv.gz'
chromatograms = ChromatogramCreator(xcms_output)

DEBUG:Chemicals:Loading 0 chromatograms
../codes\VMSfunctions\Common.py:56: RuntimeWarning:

divide by zero encountered in double_scalars

DEBUG:Chemicals:Loading 5000 chromatograms
DEBUG:Chemicals:Loading 10000 chromatograms
DEBUG:Chemicals:Loading 15000 chromatograms
DEBUG:Chemicals:Loading 20000 chromatograms
DEBUG:Chemicals:Loading 25000 chromatograms
DEBUG:Chemicals:Loading 30000 chromatograms
DEBUG:Chemicals:Loading 35000 chromatograms
DEBUG:Chemicals:Loading 40000 chromatograms
DEBUG:Chemicals:Loading 45000 chromatograms


In [8]:
chemicals = ChemicalCreator(ps, chromatograms)

# Improve Chemical Creator

Need to test the chemical creator in more situations to make sure the code works as intended

In [20]:
min_ms1_intensity = 2E5
rt_range = [(3*60, 21*60)]
mz_range = [(0, 1000)]
dataset = chemicals.sample(mz_range, rt_range, min_ms1_intensity, 2000, 2, "Known", None, hmdb, 0.1)

DEBUG:Chemicals:2000 ms1 peaks to be created.
DEBUG:Chemicals:i = 0
DEBUG:Chemicals:i = 25
DEBUG:Chemicals:i = 50
DEBUG:Chemicals:i = 75
DEBUG:Chemicals:i = 100
DEBUG:Chemicals:i = 125
DEBUG:Chemicals:i = 150
DEBUG:Chemicals:i = 175
DEBUG:Chemicals:i = 200
DEBUG:Chemicals:i = 225
DEBUG:Chemicals:i = 250
DEBUG:Chemicals:i = 275
DEBUG:Chemicals:i = 300
DEBUG:Chemicals:i = 325
DEBUG:Chemicals:i = 350
DEBUG:Chemicals:i = 375
DEBUG:Chemicals:i = 400
DEBUG:Chemicals:i = 425
DEBUG:Chemicals:i = 450
DEBUG:Chemicals:i = 475
DEBUG:Chemicals:i = 500
DEBUG:Chemicals:i = 525
DEBUG:Chemicals:i = 550
DEBUG:Chemicals:i = 575
DEBUG:Chemicals:i = 600
DEBUG:Chemicals:i = 625
DEBUG:Chemicals:i = 650
DEBUG:Chemicals:i = 675
DEBUG:Chemicals:i = 700
DEBUG:Chemicals:i = 725
DEBUG:Chemicals:i = 750
DEBUG:Chemicals:i = 775
DEBUG:Chemicals:i = 800
DEBUG:Chemicals:i = 825
DEBUG:Chemicals:i = 850
DEBUG:Chemicals:i = 875
DEBUG:Chemicals:i = 900
DEBUG:Chemicals:i = 925
DEBUG:Chemicals:i = 950
DEBUG:Chemicals:i = 975

# Test Top N Controller

In [21]:
max_rt = 10*60                  # the maximum retention time of scans to generate
N = 1                           # top-5 DDA fragmentation
mz_tol = 5                      # the mz isolation window around a selected precursor ion
rt_tol = 15                     # the rt window around a selected precursor ion to prevent it from fragmented multiple times
min_ms2_intensity = 5000        # the minimum ms2 peak intensity

mass_spec = IndependentMassSpectrometer(POSITIVE, dataset, density=ps.density_estimator)
controller = TopNController(mass_spec, N, mz_tol, rt_tol, min_ms2_intensity=min_ms2_intensity)

set_log_level_warning() # We don't want to see too many messages as the controller is running
# set_log_level_info()
# set_log_level_debug()

controller.run(max_rt)

INFO:TopNController:Acquisition open
INFO:TopNController:Received Scan 1354 num_peaks=6 rt=174.04 ms_level=1
INFO:TopNController:Received Scan 1355 num_peaks=13 rt=174.36 ms_level=2
INFO:TopNController:Received Scan 1356 num_peaks=6 rt=174.62 ms_level=1
INFO:TopNController:Received Scan 1357 num_peaks=6 rt=174.75 ms_level=1
INFO:TopNController:Received Scan 1358 num_peaks=6 rt=174.83 ms_level=1
INFO:TopNController:Received Scan 1359 num_peaks=6 rt=174.89 ms_level=1
INFO:TopNController:Received Scan 1360 num_peaks=6 rt=174.97 ms_level=1
INFO:TopNController:Received Scan 1361 num_peaks=6 rt=175.05 ms_level=1
INFO:TopNController:Received Scan 1362 num_peaks=6 rt=175.19 ms_level=1
INFO:TopNController:Received Scan 1363 num_peaks=6 rt=175.24 ms_level=1
INFO:TopNController:Received Scan 1364 num_peaks=6 rt=175.37 ms_level=1
INFO:TopNController:Received Scan 1365 num_peaks=12 rt=175.74 ms_level=1
INFO:TopNController:Received Scan 1366 num_peaks=12 rt=175.82 ms_level=1
INFO:TopNController:Rece

INFO:TopNController:Received Scan 1466 num_peaks=91 rt=190.02 ms_level=1
INFO:TopNController:Received Scan 1467 num_peaks=97 rt=190.23 ms_level=1
INFO:TopNController:Received Scan 1468 num_peaks=97 rt=190.30 ms_level=1
INFO:TopNController:Received Scan 1469 num_peaks=97 rt=190.37 ms_level=1
INFO:TopNController:Received Scan 1470 num_peaks=102 rt=190.45 ms_level=1
INFO:TopNController:Received Scan 1471 num_peaks=102 rt=190.64 ms_level=1
INFO:TopNController:Received Scan 1472 num_peaks=102 rt=190.73 ms_level=1
INFO:TopNController:Received Scan 1473 num_peaks=102 rt=190.91 ms_level=1
INFO:TopNController:Received Scan 1474 num_peaks=108 rt=191.03 ms_level=1
INFO:TopNController:Received Scan 1475 num_peaks=113 rt=191.19 ms_level=1
INFO:TopNController:Received Scan 1476 num_peaks=113 rt=191.26 ms_level=1
INFO:TopNController:Received Scan 1477 num_peaks=113 rt=191.31 ms_level=1
INFO:TopNController:Received Scan 1478 num_peaks=113 rt=191.39 ms_level=1
INFO:TopNController:Received Scan 1479 num

INFO:TopNController:Received Scan 1577 num_peaks=211 rt=204.45 ms_level=1
INFO:TopNController:Received Scan 1578 num_peaks=211 rt=204.53 ms_level=1
INFO:TopNController:Received Scan 1579 num_peaks=211 rt=204.57 ms_level=1
INFO:TopNController:Received Scan 1580 num_peaks=211 rt=204.68 ms_level=1
INFO:TopNController:Received Scan 1581 num_peaks=211 rt=204.75 ms_level=1
INFO:TopNController:Received Scan 1582 num_peaks=216 rt=204.80 ms_level=1
INFO:TopNController:Received Scan 1583 num_peaks=216 rt=204.96 ms_level=1
INFO:TopNController:Received Scan 1584 num_peaks=216 rt=205.33 ms_level=1
INFO:TopNController:Received Scan 1585 num_peaks=216 rt=205.42 ms_level=1
INFO:TopNController:Received Scan 1586 num_peaks=216 rt=205.59 ms_level=1
INFO:TopNController:Received Scan 1587 num_peaks=210 rt=205.86 ms_level=1
INFO:TopNController:Received Scan 1588 num_peaks=210 rt=205.93 ms_level=1
INFO:TopNController:Received Scan 1589 num_peaks=210 rt=206.05 ms_level=1
INFO:TopNController:Received Scan 1590

INFO:TopNController:Received Scan 1688 num_peaks=208 rt=219.31 ms_level=1
INFO:TopNController:Received Scan 1689 num_peaks=208 rt=219.44 ms_level=1
INFO:TopNController:Received Scan 1690 num_peaks=208 rt=219.51 ms_level=1
INFO:TopNController:Received Scan 1691 num_peaks=214 rt=219.57 ms_level=1
INFO:TopNController:Received Scan 1692 num_peaks=214 rt=219.64 ms_level=1
INFO:TopNController:Received Scan 1693 num_peaks=214 rt=219.79 ms_level=1
INFO:TopNController:Received Scan 1694 num_peaks=214 rt=220.15 ms_level=1
INFO:TopNController:Received Scan 1695 num_peaks=208 rt=220.22 ms_level=1
INFO:TopNController:Received Scan 1696 num_peaks=208 rt=220.31 ms_level=1
INFO:TopNController:Received Scan 1697 num_peaks=208 rt=220.42 ms_level=1
INFO:TopNController:Received Scan 1698 num_peaks=208 rt=220.68 ms_level=1
INFO:TopNController:Received Scan 1699 num_peaks=208 rt=220.77 ms_level=1
INFO:TopNController:Received Scan 1700 num_peaks=208 rt=220.89 ms_level=1
INFO:TopNController:Received Scan 1701

INFO:TopNController:Received Scan 1799 num_peaks=6 rt=233.23 ms_level=2
INFO:TopNController:Received Scan 1800 num_peaks=199 rt=233.36 ms_level=1
INFO:TopNController:Received Scan 1801 num_peaks=199 rt=233.44 ms_level=1
INFO:TopNController:Received Scan 1802 num_peaks=205 rt=233.52 ms_level=1
INFO:TopNController:Received Scan 1803 num_peaks=210 rt=233.89 ms_level=1
INFO:TopNController:Received Scan 1804 num_peaks=210 rt=233.97 ms_level=1
INFO:TopNController:Received Scan 1805 num_peaks=210 rt=234.05 ms_level=1
INFO:TopNController:Received Scan 1806 num_peaks=210 rt=234.11 ms_level=1
INFO:TopNController:Received Scan 1807 num_peaks=210 rt=234.19 ms_level=1
INFO:TopNController:Received Scan 1808 num_peaks=210 rt=234.38 ms_level=1
INFO:TopNController:Received Scan 1809 num_peaks=210 rt=234.44 ms_level=1
INFO:TopNController:Received Scan 1810 num_peaks=205 rt=234.51 ms_level=1
INFO:TopNController:Received Scan 1811 num_peaks=205 rt=234.58 ms_level=1
INFO:TopNController:Received Scan 1812 n

...

INFO:TopNController:Received Scan 4132 num_peaks=190 rt=536.43 ms_level=1
INFO:TopNController:Received Scan 4133 num_peaks=190 rt=536.50 ms_level=1
INFO:TopNController:Received Scan 4134 num_peaks=195 rt=536.70 ms_level=1
INFO:TopNController:Received Scan 4135 num_peaks=195 rt=536.82 ms_level=1
INFO:TopNController:Received Scan 4136 num_peaks=195 rt=536.96 ms_level=1
INFO:TopNController:Received Scan 4137 num_peaks=195 rt=537.04 ms_level=1
INFO:TopNController:Received Scan 4138 num_peaks=201 rt=537.41 ms_level=1
INFO:TopNController:Received Scan 4139 num_peaks=201 rt=537.54 ms_level=1
INFO:TopNController:Received Scan 4140 num_peaks=201 rt=537.66 ms_level=1
INFO:TopNController:Received Scan 4141 num_peaks=201 rt=537.74 ms_level=1
INFO:TopNController:Received Scan 4142 num_peaks=11 rt=537.81 ms_level=2
INFO:TopNController:Received Scan 4143 num_peaks=201 rt=537.93 ms_level=1
INFO:TopNController:Received Scan 4144 num_peaks=201 rt=537.99 ms_level=1
INFO:TopNController:Received Scan 4145 

INFO:TopNController:Received Scan 4243 num_peaks=249 rt=550.00 ms_level=1
INFO:TopNController:Received Scan 4244 num_peaks=249 rt=550.08 ms_level=1
INFO:TopNController:Received Scan 4245 num_peaks=249 rt=550.19 ms_level=1
INFO:TopNController:Received Scan 4246 num_peaks=254 rt=550.47 ms_level=1
INFO:TopNController:Received Scan 4247 num_peaks=237 rt=550.73 ms_level=1
INFO:TopNController:Received Scan 4248 num_peaks=231 rt=550.87 ms_level=1
INFO:TopNController:Received Scan 4249 num_peaks=231 rt=550.95 ms_level=1
INFO:TopNController:Received Scan 4250 num_peaks=231 rt=551.03 ms_level=1
INFO:TopNController:Received Scan 4251 num_peaks=231 rt=551.16 ms_level=1
INFO:TopNController:Received Scan 4252 num_peaks=231 rt=551.23 ms_level=1
INFO:TopNController:Received Scan 4253 num_peaks=225 rt=551.30 ms_level=1
INFO:TopNController:Received Scan 4254 num_peaks=225 rt=551.36 ms_level=1
INFO:TopNController:Received Scan 4255 num_peaks=225 rt=551.42 ms_level=1
INFO:TopNController:Received Scan 4256

INFO:TopNController:Received Scan 4354 num_peaks=272 rt=564.40 ms_level=1
INFO:TopNController:Received Scan 4355 num_peaks=272 rt=564.46 ms_level=1
INFO:TopNController:Received Scan 4356 num_peaks=267 rt=564.53 ms_level=1
INFO:TopNController:Received Scan 4357 num_peaks=273 rt=564.61 ms_level=1
INFO:TopNController:Received Scan 4358 num_peaks=279 rt=564.69 ms_level=1
INFO:TopNController:Received Scan 4359 num_peaks=268 rt=564.88 ms_level=1
INFO:TopNController:Received Scan 4360 num_peaks=268 rt=565.01 ms_level=1
INFO:TopNController:Received Scan 4361 num_peaks=262 rt=565.20 ms_level=1
INFO:TopNController:Received Scan 4362 num_peaks=263 rt=565.35 ms_level=1
INFO:TopNController:Received Scan 4363 num_peaks=263 rt=565.41 ms_level=1
INFO:TopNController:Received Scan 4364 num_peaks=263 rt=565.49 ms_level=1
INFO:TopNController:Received Scan 4365 num_peaks=262 rt=565.85 ms_level=1
INFO:TopNController:Received Scan 4366 num_peaks=256 rt=565.92 ms_level=1
INFO:TopNController:Received Scan 4367

INFO:TopNController:Received Scan 4465 num_peaks=212 rt=577.93 ms_level=1
INFO:TopNController:Received Scan 4466 num_peaks=212 rt=578.10 ms_level=1
INFO:TopNController:Received Scan 4467 num_peaks=213 rt=578.19 ms_level=1
INFO:TopNController:Received Scan 4468 num_peaks=213 rt=578.34 ms_level=1
INFO:TopNController:Received Scan 4469 num_peaks=213 rt=578.42 ms_level=1
INFO:TopNController:Received Scan 4470 num_peaks=204 rt=578.78 ms_level=1
INFO:TopNController:Received Scan 4471 num_peaks=204 rt=578.95 ms_level=1
INFO:TopNController:Received Scan 4472 num_peaks=204 rt=579.08 ms_level=1
INFO:TopNController:Received Scan 4473 num_peaks=204 rt=579.17 ms_level=1
INFO:TopNController:Received Scan 4474 num_peaks=210 rt=579.31 ms_level=1
INFO:TopNController:Received Scan 4475 num_peaks=210 rt=579.38 ms_level=1
INFO:TopNController:Received Scan 4476 num_peaks=210 rt=579.72 ms_level=1
INFO:TopNController:Received Scan 4477 num_peaks=210 rt=579.79 ms_level=1
INFO:TopNController:Received Scan 4478

INFO:TopNController:Received Scan 4577 num_peaks=238 rt=592.94 ms_level=1
INFO:TopNController:Received Scan 4578 num_peaks=244 rt=593.01 ms_level=1
INFO:TopNController:Received Scan 4579 num_peaks=244 rt=593.08 ms_level=1
INFO:TopNController:Received Scan 4580 num_peaks=249 rt=593.13 ms_level=1
INFO:TopNController:Received Scan 4581 num_peaks=249 rt=593.28 ms_level=1
INFO:TopNController:Received Scan 4582 num_peaks=249 rt=593.36 ms_level=1
INFO:TopNController:Received Scan 4583 num_peaks=249 rt=593.42 ms_level=1
INFO:TopNController:Received Scan 4584 num_peaks=243 rt=593.54 ms_level=1
INFO:TopNController:Received Scan 4585 num_peaks=243 rt=593.61 ms_level=1
INFO:TopNController:Received Scan 4586 num_peaks=237 rt=593.77 ms_level=1
INFO:TopNController:Received Scan 4587 num_peaks=243 rt=593.92 ms_level=1
INFO:TopNController:Received Scan 4588 num_peaks=243 rt=594.00 ms_level=1
INFO:TopNController:Received Scan 4589 num_peaks=243 rt=594.14 ms_level=1
INFO:TopNController:Received Scan 4590

In [22]:
controller.write_mzML('my_analysis2', '../models/out2.mzML')

# DIA Mass Spec

In [92]:
dia_design = "basic"
window_type = "even"
kaufmann_design = None
extra_bins = 0
num_windows=10

mass_spec = IndependentMassSpectrometer(POSITIVE, dataset, density=ps.density_estimator)
controller = TreeController(mass_spec, dia_design, window_type, kaufmann_design, extra_bins, num_windows)

set_log_level_warning() # We don't want to see too many messages as the controller is running
# set_log_level_info()
# set_log_level_debug()

controller.run(max_rt)

INFO:TreeController:Acquisition open
INFO:TreeController:Received MS1 scan Scan 1464 num_peaks=7 rt=183.59 ms_level=1
DEBUG:TreeController:Window locations [[[(-10.0, 100.0)]], [[(100.0, 200.0)]], [[(200.0, 300.0)]], [[(300.0, 400.0)]], [[(400.0, 500.0)]], [[(500.0, 600.0)]], [[(600.0, 700.0)]], [[(700.0, 800.0)]], [[(800.0, 900.0)]], [[(900.0, 1010.0)]]]
INFO:TreeController:Received MS2 scan Scan 1472 num_peaks=40 rt=185.17 ms_level=2
INFO:TreeController:Received MS1 scan Scan 1475 num_peaks=7 rt=185.60 ms_level=1
DEBUG:TreeController:Window locations [[[(-10.0, 100.0)]], [[(100.0, 200.0)]], [[(200.0, 300.0)]], [[(300.0, 400.0)]], [[(400.0, 500.0)]], [[(500.0, 600.0)]], [[(600.0, 700.0)]], [[(700.0, 800.0)]], [[(800.0, 900.0)]], [[(900.0, 1010.0)]]]
INFO:TreeController:Received MS2 scan Scan 1483 num_peaks=40 rt=186.98 ms_level=2
INFO:TreeController:Received MS1 scan Scan 1486 num_peaks=7 rt=187.67 ms_level=1
DEBUG:TreeController:Window locations [[[(-10.0, 100.0)]], [[(100.0, 200.0)]

DEBUG:TreeController:Window locations [[[(-10.0, 100.0)]], [[(100.0, 200.0)]], [[(200.0, 300.0)]], [[(300.0, 400.0)]], [[(400.0, 500.0)]], [[(500.0, 600.0)]], [[(600.0, 700.0)]], [[(700.0, 800.0)]], [[(800.0, 900.0)]], [[(900.0, 1010.0)]]]
INFO:TreeController:Received MS2 scan Scan 2445 num_peaks=40 rt=321.81 ms_level=2
INFO:TreeController:Received MS1 scan Scan 2454 num_peaks=5 rt=323.27 ms_level=1
DEBUG:TreeController:Window locations [[[(-10.0, 100.0)]], [[(100.0, 200.0)]], [[(200.0, 300.0)]], [[(300.0, 400.0)]], [[(400.0, 500.0)]], [[(500.0, 600.0)]], [[(600.0, 700.0)]], [[(700.0, 800.0)]], [[(800.0, 900.0)]], [[(900.0, 1010.0)]]]
INFO:TreeController:Received MS2 scan Scan 2456 num_peaks=40 rt=323.55 ms_level=2
INFO:TreeController:Received MS1 scan Scan 3237 num_peaks=7 rt=424.29 ms_level=1
DEBUG:TreeController:Window locations [[[(-10.0, 100.0)]], [[(100.0, 200.0)]], [[(200.0, 300.0)]], [[(300.0, 400.0)]], [[(400.0, 500.0)]], [[(500.0, 600.0)]], [[(600.0, 700.0)]], [[(700.0, 800.0

INFO:TreeController:Received MS1 scan Scan 3422 num_peaks=6 rt=455.22 ms_level=1
DEBUG:TreeController:Window locations [[[(-10.0, 100.0)]], [[(100.0, 200.0)]], [[(200.0, 300.0)]], [[(300.0, 400.0)]], [[(400.0, 500.0)]], [[(500.0, 600.0)]], [[(600.0, 700.0)]], [[(700.0, 800.0)]], [[(800.0, 900.0)]], [[(900.0, 1010.0)]]]
INFO:TreeController:Received MS1 scan Scan 3612 num_peaks=6 rt=479.99 ms_level=1
DEBUG:TreeController:Window locations [[[(-10.0, 100.0)]], [[(100.0, 200.0)]], [[(200.0, 300.0)]], [[(300.0, 400.0)]], [[(400.0, 500.0)]], [[(500.0, 600.0)]], [[(600.0, 700.0)]], [[(700.0, 800.0)]], [[(800.0, 900.0)]], [[(900.0, 1010.0)]]]
INFO:TreeController:Received MS2 scan Scan 3619 num_peaks=40 rt=481.12 ms_level=2
INFO:TreeController:Received MS1 scan Scan 3623 num_peaks=6 rt=481.77 ms_level=1
DEBUG:TreeController:Window locations [[[(-10.0, 100.0)]], [[(100.0, 200.0)]], [[(200.0, 300.0)]], [[(300.0, 400.0)]], [[(400.0, 500.0)]], [[(500.0, 600.0)]], [[(600.0, 700.0)]], [[(700.0, 800.0)

INFO:TreeController:Received MS2 scan Scan 3808 num_peaks=40 rt=513.87 ms_level=2
INFO:TreeController:Received MS1 scan Scan 3810 num_peaks=7 rt=514.29 ms_level=1
DEBUG:TreeController:Window locations [[[(-10.0, 100.0)]], [[(100.0, 200.0)]], [[(200.0, 300.0)]], [[(300.0, 400.0)]], [[(400.0, 500.0)]], [[(500.0, 600.0)]], [[(600.0, 700.0)]], [[(700.0, 800.0)]], [[(800.0, 900.0)]], [[(900.0, 1010.0)]]]
INFO:TreeController:Received MS2 scan Scan 3819 num_peaks=40 rt=515.66 ms_level=2
INFO:TreeController:Received MS1 scan Scan 3821 num_peaks=7 rt=516.07 ms_level=1
DEBUG:TreeController:Window locations [[[(-10.0, 100.0)]], [[(100.0, 200.0)]], [[(200.0, 300.0)]], [[(300.0, 400.0)]], [[(400.0, 500.0)]], [[(500.0, 600.0)]], [[(600.0, 700.0)]], [[(700.0, 800.0)]], [[(800.0, 900.0)]], [[(900.0, 1010.0)]]]
INFO:TreeController:Received MS2 scan Scan 3830 num_peaks=40 rt=517.24 ms_level=2
INFO:TreeController:Received MS1 scan Scan 3832 num_peaks=7 rt=517.43 ms_level=1
DEBUG:TreeController:Window loc

INFO:TreeController:Received MS2 scan Scan 4288 num_peaks=10 rt=584.88 ms_level=2
INFO:TreeController:Received MS2 scan Scan 4289 num_peaks=30 rt=585.02 ms_level=2
INFO:TreeController:Received MS1 scan Scan 4298 num_peaks=5 rt=586.48 ms_level=1
DEBUG:TreeController:Window locations [[[(-10.0, 100.0)]], [[(100.0, 200.0)]], [[(200.0, 300.0)]], [[(300.0, 400.0)]], [[(400.0, 500.0)]], [[(500.0, 600.0)]], [[(600.0, 700.0)]], [[(700.0, 800.0)]], [[(800.0, 900.0)]], [[(900.0, 1010.0)]]]
INFO:TreeController:Received MS2 scan Scan 4299 num_peaks=10 rt=586.56 ms_level=2
INFO:TreeController:Received MS2 scan Scan 4300 num_peaks=30 rt=586.69 ms_level=2
INFO:TreeController:Received MS1 scan Scan 4309 num_peaks=5 rt=588.42 ms_level=1
DEBUG:TreeController:Window locations [[[(-10.0, 100.0)]], [[(100.0, 200.0)]], [[(200.0, 300.0)]], [[(300.0, 400.0)]], [[(400.0, 500.0)]], [[(500.0, 600.0)]], [[(600.0, 700.0)]], [[(700.0, 800.0)]], [[(800.0, 900.0)]], [[(900.0, 1010.0)]]]
INFO:TreeController:Received M

# Create Multiple Datasets

In [ ]:
min_ms1_intensity = 2E5
rt_range = [(3*60, 21*60)]
mz_range = [(0, 1000)]
dataset = chemicals.sample(mz_range, rt_range, min_ms1_intensity, 2000, 2, "Known", None, hmdb, 0.1)

In [ ]:
class MultiSampleCreator(object):
    
    def __init__(self, original_dataset, n_samples, classes, change_probabilities, intensity_noise, change_differences):
        self.original_dataset
        self.n_samples = n_samples
        self.classes = classes
        self.change_probabilities change_probabilities
        self.change_differences = change_differences
        self.intensity_noise = intensity_noise
        
        
        
        # return samples
            # sample_classes (which class each sample belongs to)
            # chemical_status (changed or unchanged by class)
            # chemical_differences (differences in intensities from first class)
        